# Creating An MNIST Dataset Classifier
For this portion of the tutorial, please refer to [this tutorial](https://www.tensorflow.org/datasets/keras_example#step_2_create_and_train_the_model) that I have followed.

## Import Libraries

In [49]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [50]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

## Training Pipeline

In [51]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

## Testing Pipeline

In [52]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Create And Train The Model

In [53]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6


C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


469/469 [==============================] - 3s 3ms/step - loss: 0.3549 - sparse_categorical_accuracy: 0.9016 - val_loss: 0.2020 - val_sparse_categorical_accuracy: 0.9412
Epoch 2/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1630 - sparse_categorical_accuracy: 0.9536 - val_loss: 0.1404 - val_sparse_categorical_accuracy: 0.9598
Epoch 3/6
469/469 [==============================] - 1s 2ms/step - loss: 0.1159 - sparse_categorical_accuracy: 0.9668 - val_loss: 0.1093 - val_sparse_categorical_accuracy: 0.9683
Epoch 4/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0912 - sparse_categorical_accuracy: 0.9735 - val_loss: 0.0904 - val_sparse_categorical_accuracy: 0.9728
Epoch 5/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0735 - sparse_categorical_accuracy: 0.9791 - val_loss: 0.0818 - val_sparse_categorical_accuracy: 0.9749
Epoch 6/6
469/469 [==============================] - 1s 2ms/step - loss: 0.0609 - sparse_categorical_accuracy: 0.98

## Saving The Model

In [55]:
model.save('mnist_model')

INFO:tensorflow:Assets written to: mnist_model\assets


INFO:tensorflow:Assets written to: mnist_model\assets


## Running The Saved Model To Make An Inference

In [1]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def normalize_img(image):
  return tf.cast(image, tf.float32) / 255.0

In [3]:
model = tf.keras.models.load_model('mnist_model')
image = tf.keras.preprocessing.image.load_img('mnist_image_png.png')
image_tensor = tf.keras.preprocessing.image.img_to_array(image)
image_tensor_gray = tf.image.rgb_to_grayscale(image_tensor)
image_tensor_gray = tf.reshape(image_tensor_gray, [1, 28, 28])
image_tensor_gray = normalize_img(image_tensor_gray)

In [4]:
result = model.predict(image_tensor_gray)

1/1 [==============================] - 0s 89ms/step


In [5]:
print(result.shape)

(1, 10)


In [6]:
print(f"Guess: {result[0].argmax()}\tConfidence: {result[0][result[0].argmax()]*100:.3f}%")

Guess: 0	Confidence: 99.999%
